## Robust Portfolio Optimization

Robust portfolio optimization is a variant of traditional portfolio optimization. The objective is to allocate a total investment amount across different assets to maximize the portfolio's expected return, while ensuring a very low probability (e.g., 0.5%) that the actual return falls below this expected value. This approach addresses the uncertainty inherent in portfolio returns, making it a robust optimization problem.


Consider an example where we have n = 200 assets. Let $r_i$ denote the return of the $i$-th asset. The return on Asset #200 $r_{n}=1.05$ has zero variability. The returns of the remaining assets $r_i$, $\forall i \in [n - 1]$, are random variables taking values in the intervals $[\mu_i - \sigma_i ,\mu_i + \sigma_i  ]$. $\vec{\mu}$ and $\vec{\sigma}$ are defined as:

$$ \mu_i = 1.05 + \frac{0.3\left(n - i\right)}{n - 1} , \space \sigma_i = 0.05 + \frac{0.6\left(n - i\right)}{n - 1}, \space \forall i \in [n - 1] $$



To solve this problem, we first import the required packages and generate the data. 

In [1]:
import lropt
import cvxpy as cp
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [2]:
n = 200
b1 = 1.05
b2 = 0.3
b3 = 0.05
b4 = 0.6

mu = np.zeros(n)
for i in range(1, n + 1):
    mu_i = b1 + b2 * (n - i) / (n - 1)
    mu[i - 1] = mu_i  

sigma = np.zeros(n)
for i in range(1, n + 1):
    sigma_i = b3 + b4 * (n - i) / (n - 1)
    sigma[i - 1] = sigma_i 

r = np.zeros(n)
for i in range(n):
    r_i = np.random.uniform(mu[i] - sigma[i], mu[i] + sigma[i])
    r[i] = r_i

The problem we want to solve is the uncertain Linear Optimization problem:

$$ 
\begin{aligned}
& \text{maximize} \quad t\\
& \text{subject to} \\
& \mu^T \vec{x} - \sigma^T \vec{x} + 1.05x_{n} \geq t , \\
& 1^T\vec{x}= 1, \\
& \vec{x} \geq 0
\end{aligned}
$$

$ x_i $ is the capital to be invested in asset $i$. The guarenteed yearly return is 1.05, and the worst possible return is $ r_i = \mu_i - \sigma_i $ for the uncertain returns. The robust optimal solution is *guarenteed* to keep the initial capital in the bank.




The uncertain data are the retuns $r_i = \mu_i + \sigma_i z_i, i \in [199]$ where $z_i, i\in[199]$, are independant random variables with zero mean varying in the segments $[-1, 1]$.

The robust counterpart to the initial optimization problem is:


$$ 
\begin{aligned}
& \text{maximize} \quad t\\
& \text{subject to} \\
& \mu^T \vec{x} - \sigma^T \vec{x} + 1.05x_{n} \geq t , \quad \forall z \in \mathcal{Z} \\
& 1^T\vec{x}= 1, \\
& \vec{x} \geq 0
\end{aligned}
$$

for a variety of uncertainty sets $ \mathcal{Z} $. 


In [3]:
t = cp.Variable()
x = cp.Variable(n, nonneg=True)

In the following snippet, we solve this problem using three uncertainty sets and compare the results.

In [4]:
uncertainty_sets = [lropt.Ellipsoidal(), lropt.Budget(), lropt.Box()]
names = ['ellipsoidal', 'budget', 'box']
num_dec = 3

for uc in enumerate(uncertainty_sets):
    t = cp.Variable()
    x = cp.Variable(n, nonneg=True)
    uncertainty_param = lropt.UncertainParameter(n, uncertainty_set=uc[1])

    constraints = [
        cp.sum(cp.multiply((cp.multiply(uncertainty_param,sigma) + mu), x)) + b1 * x[n - 1] >= t,
        cp.sum(x) == 1,
        x >= 0
    ]

    objective = cp.Maximize(t)

    robust_prob = lropt.RobustProblem(objective, constraints)
    print(f"The robust optimal value using {names[uc[0]]} uncertainty is {round(robust_prob.solve(), num_dec)}")


The robust optimal value using ellipsoidal uncertainty is 1.007
The robust optimal value using budget uncertainty is 1.013
The robust optimal value using box uncertainty is 1.0
